## Desafío entregable #13: CrossValidation y mejora de modelos

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from shapely.geometry import Point
import time

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import KNNImputer
from sklearn import preprocessing
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold, GridSearchCV, HalvingGridSearchCV, HalvingRandomSearchCV, RandomizedSearchCV, train_test_split

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

En este desafío se trabajará con el dataset "California Housing", el cual resumen información inmobiliaria de dicho estado de los EE.UU. Dicha información esta agrupada en "block groups" (BG), el cual es la unidad geográfica más pequeña que el U.S. Census Bureau registra informacion. Otra agrupación presente en el dataset es "household" (HH), que es el número de personas residentes en una casa. Un block group típicamente vive entre 600 y 3000 personas. Las variables presentes en el dataset son:

- longitude: longitud del BG.
- latitude: latitud del BG.
- housing_median_age: mediana de la edad del BG.
- total_rooms: promedio de ambientes por HH. 
- total_bedrooms: promedio de habitaciones por HH.
- population: población del BG.
- households: número de HH por BG. 
- median_income: mediana del ingreso del BG.
- median_house_value: mediana del valor del HH.  
- ocean_proximity: cercanía al océano, variable categórica.
- gender: género mayoritario del BG.

El objetivo es predecir la mediana del valor de los inmuebles, expresados en cientos de miles de dólares (US$ 100.000). Esta entrega hace foco en la validación cruzada y en el ajuste de hiperparámetros como mecanismo para la mejora de los modelos de aprendizaje autómatico.

El procedimiento de limpieza, curado y estandarizacion/normalización de los datos es el mismo de la entrega anterior y no se los describirá en detalle.

In [2]:
df = pd.read_csv('Housing_Price.csv')

## Repaso del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20382 non-null  float64
 3   total_rooms         20640 non-null  int64  
 4   total_bedrooms      15758 non-null  float64
 5   population          20596 non-null  float64
 6   households          19335 non-null  object 
 7   median_income       17873 non-null  float64
 8   median_house_value  20640 non-null  int64  
 9   ocean_proximity     20640 non-null  object 
 10  gender              16620 non-null  object 
dtypes: float64(6), int64(2), object(3)
memory usage: 1.7+ MB


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
longitude,20640.0,-119.569704,2.003532,-124.3500,-121.8000,-118.4900,-118.0100,-114.3100
latitude,20640.0,35.631861,2.135952,32.5400,33.9300,34.2600,37.7100,41.9500
housing_median_age,20382.0,28.676283,12.589284,1.0000,18.0000,29.0000,37.0000,52.0000
total_rooms,20640.0,2635.763081,2181.615252,2.0000,1447.7500,2127.0000,3148.0000,39320.0000
total_bedrooms,15758.0,539.920104,419.834171,1.0000,296.0000,435.0000,652.0000,6210.0000
population,20596.0,1424.928724,1132.237768,3.0000,787.0000,1166.0000,1725.0000,35682.0000
median_income,17873.0,3.939403,1.943517,0.4999,2.5986,3.5871,4.8304,15.0001
median_house_value,20640.0,206855.816909,115395.615874,14999.0000,119600.0000,179700.0000,264725.0000,500001.0000


In [5]:
df.describe(include = "object").T

,count,unique,top,freq
households,19335,1703,no,3080
ocean_proximity,20640,5,<1H OCEAN,9136
gender,16620,2,female,8673


## Limpieza de datos

In [6]:
df['households'].replace("no", np.nan , inplace=True)
df['households'] = df['households'].astype('float64')

### Tratamiento de datos tipo NaN

In [7]:
df.drop(['gender'], axis = 1, inplace = True)

In [8]:
ord = preprocessing.OrdinalEncoder()
df['ocean_proximity'] = ord.fit_transform(df[['ocean_proximity']])

In [9]:
imputer = KNNImputer(n_neighbors = 5, weights = "uniform")
df2 = pd.DataFrame(imputer.fit_transform(df), columns = df.columns)

## Curación de datos

In [10]:
coastal_shapefile_path = "ne_110m_coastline/ne_110m_coastline.shp"
coastal_data = gpd.read_file(coastal_shapefile_path)

In [11]:
df2['distance_coast'] = None

for index, row in df2.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    
    point = Point(longitude, latitude)
    distances = coastal_data.distance(point) # Calculamos la distancia a todos los puntos de la costa
    distances_min = distances.min() # Encontramos el mínimo valor de distancia

    df2.at[index, 'distance_coast'] = distances_min * 111.0445 
    
df2['distance_coast'] = df2['distance_coast'].astype('float64')

C:\Users\mauri\AppData\Local\Temp\ipykernel_6636\1402484739.py:8: UserWarning: Geometry is in a geographic CRS. Results from 'distance' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  distances = coastal_data.distance(point) # Calculamos la distancia a todos los puntos de la costa


In [12]:
los_angeles = [34.1141, -118.4068] # Longitud y latitud
san_diego = [32.8313, -117.1222]
san_jose = [37.3012, -121.848]
san_francisco = [37.7558, -122.4449]
fresno = [36.783, -119.7939]

cities = [los_angeles, san_diego, san_jose, san_francisco, fresno]
names = ['los_angeles', 'san_diego', 'san_jose', 'san_francisco', 'fresno']

In [13]:
for i in range(len(cities)):
    
    df2[f'distance_{names[i]}'] = None

    for index, row in df.iterrows():
        latitude = row['latitude']
        longitude = row['longitude']
        
        point1 = Point(cities[i][1], cities[i][0])
        point2 = Point(longitude, latitude)
        
        distance = point1.distance(point2)
        
        df2.at[index, f'distance_{names[i]}'] = distance * 111.0445 
        
    df2[f'distance_{names[i]}'] = df2[f'distance_{names[i]}'].astype('float64')

In [14]:
df2.drop(['ocean_proximity', 'latitude', 'longitude'], axis = 1, inplace = True)

## Reducción de dimensionalidad

Se reducirá el número de variables a través del método de importancia, empleando bosques de decisión. Este método probó ser el más rápido y por ello se lo selecciona

In [15]:
pt = PowerTransformer()

df3 = df2.copy()
df4 = pd.DataFrame(pt.fit_transform(df3), columns = df3.columns)

In [16]:
X = df4.drop(['median_house_value'], axis = 1) 
y = df4['median_house_value']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

regr = RandomForestRegressor(max_depth = 3, random_state = 0)

regr.fit(X_train, y_train)
impor_variable = regr.feature_importances_

In [17]:
importancias = pd.DataFrame({'feature': list(X.columns), 'importance': impor_variable}).sort_values('importance', ascending = False)
importancias

,feature,importance
6,distance_coast,0.521505
5,median_income,0.474019
11,distance_fresno,0.004362
10,distance_san_francisco,0.000114
0,housing_median_age,0.000000
1,total_rooms,0.000000
2,total_bedrooms,0.000000
3,population,0.000000
4,households,0.000000
7,distance_los_angeles,0.000000


In [18]:
impor_nulas = list(importancias[importancias['importance'] == 0.0]['feature'])
impor_nulas

['housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'distance_los_angeles',
 'distance_san_diego',
 'distance_san_jose']

Estas son las variables que descartaremos, dado que no parecen contribuir a la predición de la mediana del valor de los inmuebles.

In [19]:
X = X.drop(columns = impor_nulas)
X.head()

,median_income,distance_coast,distance_san_francisco,distance_fresno
0,1.926231,-0.098137,-1.602920,-0.314668
1,1.920074,-0.087038,-1.603219,-0.331393
2,1.628791,-0.152815,-1.623067,-0.317584
3,1.073383,-0.183544,-1.631276,-0.308713
4,0.211877,-0.183544,-1.631276,-0.308713


Este el dataset resultante, con las variables que tienen importancia distinta de cero y que serán utilizadas en la terna entrenamiento - validación - testeo.

## Mejora de modelos de aprendizaje automático

Como dice el título de la sección, se buscarán mejorar los modelos mediante validación cruzada y por medio del ajuste fino de los hiperparámetros mediante el uso de diversos algoritmos. El primero que veremos es la validación cruzada, puedo que luego el ajuste fino de los hiperparámetros hace uso extensivo de la misma.

### Validación cruzada

Se empleará el algoritmo de validación cruzada KFold, en donde se particiona el trainset en k partes, quedando una de ellas como validación y las restantes de entrenamiento. Este algoritmo puede ser potencialmente muy lento, es por ello que se buscará una alternativa más rápida a XGBoost, que fue el algoritmo que mejor se desempeñó en entregas pasadas. Dicha alternativa es el algoritmo LightGBM, que también es un modelo de Boosting, pero que se caracteriza por su velocidad de cómputo.

In [20]:
xgb = XGBRegressor()
start_time = time.time()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

end_time = time.time()
execution_time = end_time - start_time

MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
tiempo = execution_time

print(f'''Las métricas obtenidas para XGBoost son:
      MAE = {MAE.round(4)}
      MSE = {MSE.round(4)}
      R2 = {r2.round(4)}
      Tiempo = {np.round(tiempo,4)} s''')

Las métricas obtenidas para XGBoost son:
      MAE = 0.2832
      MSE = 0.1642
      R2 = 0.837
      Tiempo = 1.0014 s


In [21]:
lgbm = LGBMRegressor()

start_time = time.time()

lgbm.fit(X_train, y_train)
y_pred = lgbm.predict(X_test)

end_time = time.time()
execution_time = end_time - start_time

MAE = mean_absolute_error(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
tiempo = execution_time

print(f'''Las métricas obtenidas para LightGMB son:
      MAE = {MAE.round(4)}
      MSE = {MSE.round(4)}
      R2 = {r2.round(4)}
      Tiempo = {np.round(tiempo,4)} s''')

Las métricas obtenidas para LightGMB son:
      MAE = 0.2878
      MSE = 0.1639
      R2 = 0.8373
      Tiempo = 0.1659 s


Se observa que ambos algortimos poseen métricas de bondad de ajuste muy similares, sin embargo, XGBoost tarda aprox. 1 s, en tanto que con LightGMB este tiempo es de sólo 0.2 s, sólo una quinta parte.

In [22]:
kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

model = LGBMRegressor()
score = cross_val_score(model, X, y, cv = kf, scoring = 'r2')

print(f'R2 de cada fold = {score.round(4)}')
print(f'R2 promedio = {"{:.4f}".format(score.mean())}')

R2 de cada fold = [0.8404 0.8298 0.8509 0.8362 0.8388 0.8308 0.8502 0.8481 0.8281 0.8337]
R2 promedio = 0.8387


Al particionar el trainset en k = 10 folds y luego aplicar el modelo a cada uno de estos fold se obtiene luego la métrica R2 promedio. Este valor es muy cercano al obtenido en la aplicación del modelo en el trainset al completo. Esto indica que el modelo aplicado (con todos los hiperparámetros en valores por defecto) hace un buen ajuste.

### Ajuste manual

El trabajo de ajuste fino de los parámetros puede hacerse de forma manual a través del mismo algoritmo cross_val_score. Inspeccionaremos diversos hiperparámetros y vamos eligiendo secuencialmente el que mejor valor de R2 ofrece, así hasta llegar al modelo final.

In [23]:
boosting_type = ['gbdt', 'dart']

for boosting in boosting_type:
    model = LGBMRegressor(boosting_type = boosting)
    score = cross_val_score(model, X, y, cv = kf, scoring = 'r2')
    print(f'R2 promedio tipo de boosting: {boosting} = {"{:.4f}".format(score.mean())}')

R2 promedio tipo de boosting: gbdt = 0.8387
R2 promedio tipo de boosting: dart = 0.8053


In [24]:
max_depth = [1, 2, 3, 4, 5, -1]

for depth in max_depth:
    model = LGBMRegressor(boosting_type = 'gbdt', max_depth = depth)
    score = cross_val_score(model, X, y, cv = kf, scoring = 'r2')
    print(f'R2 promedio profundidad máxima: {depth} = {"{:.4f}".format(score.mean())}')

R2 promedio profundidad máxima: 1 = 0.7274
R2 promedio profundidad máxima: 2 = 0.7677
R2 promedio profundidad máxima: 3 = 0.7910
R2 promedio profundidad máxima: 4 = 0.8094
R2 promedio profundidad máxima: 5 = 0.8243
R2 promedio profundidad máxima: -1 = 0.8387


In [25]:
learning_rate = [0.001, 0.01, 0.1, 1]

for rate in learning_rate:
    model = LGBMRegressor(boosting_type = 'gbdt', max_depth = -1, learning_rate = rate)
    score = cross_val_score(model, X, y, cv = kf, scoring = 'r2')
    print(f'R2 promedio tasa de aprendizaje: {rate} = {"{:.4f}".format(score.mean())}')

R2 promedio tasa de aprendizaje: 0.001 = 0.1331
R2 promedio tasa de aprendizaje: 0.01 = 0.6581
R2 promedio tasa de aprendizaje: 0.1 = 0.8387
R2 promedio tasa de aprendizaje: 1 = 0.8147


In [26]:
n_estimators = [10, 50, 100, 500]

for n in n_estimators:
    model = LGBMRegressor(boosting_type = 'gbdt', max_depth = -1, learning_rate = 0.1, n_estimators = n)
    score = cross_val_score(model, X, y, cv = kf, scoring = 'r2')
    print(f'R2 promedio número de estimadores: {n} = {"{:.4f}".format(score.mean())}')

R2 promedio número de estimadores: 10 = 0.6677
R2 promedio número de estimadores: 50 = 0.8229
R2 promedio número de estimadores: 100 = 0.8387
R2 promedio número de estimadores: 500 = 0.8559


Luego de ajustar diversos hiperparámetros, seleccionamos los siguientes como definitivos boosting_type = 'gbdt', max_depth = -1, learning_rate = 0.1, n_estimators = 500 y hacemos una prueba rápida de overfitting.

In [27]:
def metricas(y, y_pred):
    mae = mean_absolute_error(y, y_pred)
    mse = mean_squared_error(y, y_pred)
    r2 = r2_score(y, y_pred)
    
    return mae, mse, r2

In [28]:
model = LGBMRegressor(boosting_type = 'gbdt', max_depth = -1, learning_rate = 0.1, n_estimators = 500)
model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

mae_train, mse_train, r2_train = metricas(y_train, y_pred_train)
mae_test, mse_test, r2_test = metricas(y_test, y_pred_test)

comparacion = pd.DataFrame({'Dataset': ['Train', 'Test'], 
                            'MAE': [mae_train, mae_test], 
                            'MSE': [mse_train, mse_test], 
                            'R2': [r2_train, r2_test]})
comparacion.round(4)

,Dataset,MAE,MSE,R2
0,Train,0.2051,0.0813,0.9185
1,Test,0.2680,0.1492,0.8518


Con una prueba rápida observamos que las métricas en el testset son consistentemente menores que en el trainset, indicando que el modelo no está haciendo overfitting.

Existen métodos automáticos para realizar esta exploración de hiperparámetros, probaremos varios de ellos y realizaremos una comparación.

### Grid Search

Grid Search automatiza la busqueda de los hiperparámetros, haciendo una grilla de todas las combinaciones posibles de los hiperparámetros solicitados, que se introducen en el algoritmo GridSearchCV como un diccionario. GridSearchCV luego selecciona el conjunto de hiperparámetros que mejores métricas ofrece. 

In [29]:
params_grid = {
        'boosting_type': ['gbdt', 'dart'],
        'max_depth': [1, 2, 3, 4, 5, -1],
        'learning_rate': [0.001, 0.01, 0.1, 1],
        'n_estimators': [10, 50, 100, 500],
        }

model = LGBMRegressor()
start_time = time.time()

grid_cv = GridSearchCV(model, params_grid, scoring = 'r2', n_jobs=-1, cv = 5)
grid_cv.fit(X_train, y_train)

end_time = time.time()
time_grid_cv = end_time - start_time

print("Mejores Parametros", grid_cv.best_params_)
print("Mejor CV score", grid_cv.best_score_)
print(f'R2 del modelo = {round(r2_score(y_test, grid_cv.predict(X_test)), 4)}')

Mejores Parametros {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 500}
Mejor CV score 0.8492220791164536
R2 del modelo = 0.8518


Con esta búsqueda automática arribamos a los hiperparámetros que más alto valor de R2, que son los mismos que se encontraron por medio de la búsqueda manual.

### Randomized Search CV

Este algoritmo prueba un número fijo de combinaciones de hiperparámetros (n_iter = 10) al azar, es decir, no prueba todas las posibles combinaciones pero igualmente informa la que mejor resultados da.

In [30]:
start_time = time.time()

rgrid_cv = RandomizedSearchCV(model, params_grid, scoring = 'r2', n_jobs=-1, cv = 5)
rgrid_cv.fit(X_train, y_train)

end_time = time.time()
time_rgrid_cv = end_time - start_time

print("Mejores parametros", rgrid_cv.best_params_)
print("Mejor score de CV", rgrid_cv.best_score_)
print(f'Accuracy del modelo = {round(r2_score(y_test, rgrid_cv.predict(X_test)), 4)}')

Mejores parametros {'n_estimators': 500, 'max_depth': -1, 'learning_rate': 0.1, 'boosting_type': 'gbdt'}
Mejor score de CV 0.8492220791164536
Accuracy del modelo = 0.8518


### Halving Grid Search

Halving Grid Search opera por división sucesiva por mitades. Es un proceso de selección iterativo en el que todas las combinaciones de parámetros se evalúan con una pequeña cantidad de recursos en la primera iteración. Sólo algunos de estas combinaciones se seleccionan para la siguiente iteración, a la que se asignarán más recursos. Para el ajuste de parámetros, el recurso suele ser el número de muestras de entrenamiento, pero pueden ser otros.

In [31]:
start_time = time.time()

halving_cv = HalvingGridSearchCV(model, params_grid, scoring = 'r2', n_jobs=-1, factor = 3, cv = 5)
halving_cv.fit(X_train, y_train)

end_time = time.time()
time_halving_cv = end_time - start_time

print("Mejores parametros", halving_cv.best_params_)
print("Mejor Score CV", halving_cv.best_score_)
print(f'Accuracy del modelo = {round(r2_score(y_test, halving_cv.predict(X_test)), 4)}')

Mejores parametros {'boosting_type': 'gbdt', 'learning_rate': 0.1, 'max_depth': -1, 'n_estimators': 100}
Mejor Score CV 0.8363497204447796
Accuracy del modelo = 0.8373


### Halving Randomized Search

Halving Randomized Search opera similar que su similar "no aleatorio" sin embargo sólo algunas combinaciones de hiperparámetros son seleccionadas al azar,  y éste es el punto de partida en el que el proceso iterativo comienza.

In [37]:
start_time = time.time()

rhalving_cv = HalvingRandomSearchCV(model, params_grid, scoring = 'r2', n_jobs=-1, factor = 3, cv = 5)
rhalving_cv.fit(X_train, y_train)

end_time = time.time()
time_rhalving_cv = end_time - start_time

print("Mejores parametros", rhalving_cv.best_params_)
print("Mejor CV score", rhalving_cv.best_score_)
print(f'Accuracy del modelo = {round(r2_score(y_test, rhalving_cv.predict(X_test)), 4)}')

d:\Programas\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 192 is smaller than n_iter=1548. Running 192 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mejores parametros {'n_estimators': 500, 'max_depth': -1, 'learning_rate': 0.01, 'boosting_type': 'gbdt'}
Mejor CV score 0.7888642540342572
Accuracy del modelo = 0.8229


### Comparación de métodos de búsqueda de hiperparámetros

In [38]:
hyper = pd.DataFrame({'Algoritmo': ['Grid Search','Randomized Grid Search','Halving Grid Search','Randomized Halving Grid Search'], 
                            'Max. R2': [grid_cv.best_score_, rgrid_cv.best_score_, halving_cv.best_score_, rhalving_cv.best_score_], 
                            'Tiempo': [time_grid_cv, time_rgrid_cv, time_halving_cv, time_rhalving_cv]
                            })
hyper.round(4)

,Algoritmo,Max. R2,Tiempo
0,Grid Search,0.8492,172.7762
1,Randomized Grid Search,0.8492,6.8767
2,Halving Grid Search,0.8363,43.4511
3,Randomized Halving Grid Search,0.7889,5.7125


Comparando los diversos métodos, encontramos que todos los métodos excepto Randomized Halving Grid Search obtuvieron métricas similares, ésta última ofreció valores bajos. Es importante considerar el tiempo de cómputo, Grid Search es el que más demora, al analizar todas las posibles combinaciones. Por otro lado Halving Grid Search disminuye este tiempo considerablemente. Por último Randomized Grid Search y Randomized Halving Grid Search tardaron menos de diez segundos, demostrando ser súmamente rápidos. Hay que recalcar que la naturaleza random de los métodos random hace que ofrezcan resultados diferentes y tarden tiempos distintos cada vez que se los ejecuta.